## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,41.11,70,20,34.52,few clouds
1,1,Praia,CV,14.9215,-23.5087,79.97,82,85,3.53,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,40.66,81,75,16.11,moderate rain
3,3,Jamestown,US,42.0970,-79.2353,72.72,73,100,5.75,overcast clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,70.56,94,83,7.00,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Praia,CV,14.9215,-23.5087,79.97,82,85,3.53,overcast clouds
3,3,Jamestown,US,42.0970,-79.2353,72.72,73,100,5.75,overcast clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,70.56,94,83,7.00,broken clouds
6,6,Vaini,TO,-21.2000,-175.2000,78.96,83,20,9.22,few clouds
8,8,Albany,US,42.6001,-73.9662,76.59,65,100,1.61,overcast clouds
15,15,Kalaleh,IR,37.3807,55.4916,76.60,88,100,9.22,overcast clouds
17,17,Puerto Escondido,MX,15.8500,-97.0667,79.63,73,75,5.75,broken clouds
19,19,Kodiak,US,57.7900,-152.4072,63.91,52,0,11.50,clear sky
20,20,Zakharovo,RU,54.3660,39.2792,69.03,71,45,6.20,scattered clouds
21,21,Tabou,CI,4.4230,-7.3528,75.13,77,91,10.94,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                329
City                   329
Country                329
Lat                    329
Lng                    329
Max Temp               329
Humidity               329
Cloudiness             329
Wind Speed             329
Current Description    329
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Praia,CV,79.97,overcast clouds,14.9215,-23.5087,
3,Jamestown,US,72.72,overcast clouds,42.0970,-79.2353,
4,Puerto Ayora,EC,70.56,broken clouds,-0.7393,-90.3518,
6,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
8,Albany,US,76.59,overcast clouds,42.6001,-73.9662,
15,Kalaleh,IR,76.60,overcast clouds,37.3807,55.4916,
17,Puerto Escondido,MX,79.63,broken clouds,15.8500,-97.0667,
19,Kodiak,US,63.91,clear sky,57.7900,-152.4072,
20,Zakharovo,RU,69.03,scattered clouds,54.3660,39.2792,
21,Tabou,CI,75.13,overcast clouds,4.4230,-7.3528,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
#https://www.w3schools.com/python/pandas/ref_df_iterrows.asp
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = ["lat"]

    lng = ["lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [32]:
hotels = requests.get(base_url, params=params).json() 
hotels

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [10]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
